<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=a95d89a119a99d58d5f1e0ae24a85f43d63ecb24883e2ff85906c6e61b650104
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-01 14:47:55
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.43 C
-------------------
Today PnL: -37.96 K (-0.27%)
Current PnL: -23.05 L (-15.1%)
CY Booked + Current PnL: -9.16 L (-6.0%)
-------------------
Total profit:  1.60 L
Total loss:  -24.65 L
-------------------
Total Booked + Current PnL: 17.82 L (14.18%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.73%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 88.46 L (61.97%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.2%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.27,-8.57,9.39,0.02,14986.0,-14959.0,159600.0,143.30,67.0,M-SC,11.74,234.0,-1.00,1.12,38.82,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.42,-16.68,20.12,0.09,16895.0,-16806.0,83971.0,94.88,37.0,M-SC,3.21,253.0,-0.99,0.59,10.32,OX40N,NTT,DURABLES
43,ITC,452.00,-0.15,-1.38,11.98,10.44,23647.0,-2753.0,197385.0,-41.15,45.0,X-LC,1.32,5.0,-0.12,1.38,4.11,X40,NTT,FMCG
35,ICICIGI,2252.93,0.80,5.97,13.42,20.19,24790.0,10406.0,184726.0,-16.96,48.0,X-MC,7.15,68.0,0.42,1.29,21.99,X40,ATH,INSURANCE
83,VOLTAS,1530.00,-1.13,6.45,12.44,19.69,25391.0,12363.0,204105.0,-3.98,47.0,X-MC,3.22,78.0,0.49,1.43,13.68,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,IEX,219.00,5.20,-0.56,49.46,48.63,98347.0,-1113.0,198841.0,-33.87,63.0,H-SC,15.41,136.0,-0.01,1.39,10.93,XR,NTT,MISC
81,VALIANTORG,838.00,4.46,-45.25,218.09,74.15,195350.0,-74032.0,89573.0,-260.95,37.0,H-SC,18.17,149.0,-0.38,0.63,14.82,XR,NTT,CHEMICALS
76,TMPV,600.00,1.70,-14.02,65.36,42.18,155339.0,-38743.0,237667.0,-23.08,35.0,X-LC,2.30,3.0,-0.25,1.67,2.95,XY24,NTT,AUTO
13,BSOFT,831.70,1.34,-30.15,115.86,50.79,113835.0,-42401.0,98252.0,-10.32,55.0,H-SC,6.52,151.0,-0.37,0.69,14.40,XR,ATH,IT
50,LTIM,7201.88,1.06,12.63,16.89,31.66,46827.0,31094.0,277245.0,8.32,69.0,H-LC,13.99,43.0,0.66,1.94,53.83,X200,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.00,-6.62,-19.77,126.75,81.92,116738.0,-22697.0,92101.0,-52.97,20.0,M-SC,2.95,236.0,-0.19,0.65,9.03,XR,NTT,DURABLES
27,GLAXO,3466.20,-2.78,0.28,38.76,39.15,77458.0,556.0,199840.0,-19.47,41.0,X-MC,10.56,60.0,0.01,1.40,29.36,X40,ATH,PHARMA
28,GREENPANEL,537.00,-2.52,-35.25,121.06,43.14,143511.0,-64533.0,118545.0,165.50,27.0,M-SC,6.35,240.0,-0.45,0.83,11.51,XY24,NTT,MISC
54,PAGEIND,51605.07,-2.45,-8.68,38.06,26.07,56907.0,-14220.0,149520.0,-32.77,28.0,X-MC,9.94,55.0,-0.25,1.05,0.00,X40,ATH,APPARELS
11,BATAINDIA,2096.00,-2.35,-37.06,114.82,35.20,92985.0,-47687.0,80983.0,6.14,19.0,X-SC,13.68,92.0,-0.51,0.57,0.00,X40,NTT,FOOTWEAR


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,0.66,-0.08,106.89,106.72,131727.0,-100.0,123236.0,-50.44,39.0,H-SC,2.39,139.0,-0.00,0.86,23.28,AR,ATH,MISC
37,IEX,219.00,5.20,-0.56,49.46,48.63,98347.0,-1113.0,198841.0,-33.87,63.0,H-SC,15.41,136.0,-0.01,1.39,10.93,XR,NTT,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.44,-4.70,118.16,107.91,166261.0,-6936.0,140708.0,-22.58,37.0,M-SC,10.83,216.0,-0.04,0.99,0.62,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.27,-8.57,9.39,0.02,14986.0,-14959.0,159600.0,143.30,67.0,M-SC,11.74,234.0,-1.00,1.12,38.82,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.42,-16.68,20.12,0.09,16895.0,-16806.0,83971.0,94.88,37.0,M-SC,3.21,253.0,-0.99,0.59,10.32,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.60,-22.34,46.11,13.47,96568.0,-60237.0,209430.0,-68.12,34.0,H-SC,1.76,158.0,-0.62,1.47,6.49,XY24,NTT,PAINTS
67,SIS,528.00,-1.70,-25.67,63.14,21.26,52110.0,-28501.0,82531.0,1945.08,41.0,H-SC,3.75,166.0,-0.55,0.58,11.51,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,0.32,2.67,67.78,72.27,117923.0,4531.0,173979.0,-11.98,60.0,M-LC,3.5,99.0,0.04,1.22,9.01,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,0.32,2.67,67.78,72.27,117923.0,4531.0,173979.0,-11.98,60.0,M-LC,3.50,99.0,0.04,1.22,9.01,XR,NTT,IT
38,INDIAMART,4810.62,0.66,-0.08,106.89,106.72,131727.0,-100.0,123236.0,-50.44,39.0,H-SC,2.39,139.0,-0.00,0.86,23.28,AR,ATH,MISC
37,IEX,219.00,5.20,-0.56,49.46,48.63,98347.0,-1113.0,198841.0,-33.87,63.0,H-SC,15.41,136.0,-0.01,1.39,10.93,XR,NTT,MISC
86,ZYDUSLIFE,1286.17,-0.74,-1.35,37.48,35.63,77488.0,-2828.0,206746.0,-15.41,41.0,H-MC,3.71,119.0,-0.04,1.45,15.10,AR,ATH,PHARMA
25,FINCABLES,1641.55,0.44,-4.70,118.16,107.91,166261.0,-6936.0,140708.0,-22.58,37.0,M-SC,10.83,216.0,-0.04,0.99,0.62,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-2.35,-37.06,114.82,35.20,92985.0,-47687.0,80983.0,6.14,19.0,X-SC,13.68,92.0,-0.51,0.57,0.00,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-0.59,-14.57,73.77,48.45,126367.0,-29208.0,171298.0,-20.73,28.0,X-MC,7.77,64.0,-0.23,1.20,0.00,X40N,ATH,FINANCE
54,PAGEIND,51605.07,-2.45,-8.68,38.06,26.07,56907.0,-14220.0,149520.0,-32.77,28.0,X-MC,9.94,55.0,-0.25,1.05,0.00,X40,ATH,APPARELS
23,DMART,5391.45,-1.10,-6.07,36.40,28.12,71939.0,-12772.0,197635.0,-19.00,33.0,X-LC,4.53,19.0,-0.18,1.38,16.12,X40N,ATH,FMCG
46,JSWINFRA,342.00,0.68,-8.29,25.64,15.23,47110.0,-16612.0,183735.0,-21.45,34.0,X-MC,6.89,66.0,-0.35,1.29,21.29,X40N,NTT,REALTY


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.0,-0.15,-1.38,11.98,10.44,23647.0,-2753.0,197385.0,-41.15,45.0,X-LC,1.32,5.0,-0.12,1.38,4.11,X40,NTT,FMCG
3,ACC,3906.0,-0.06,-22.27,111.19,64.15,205646.0,-53001.0,184950.0,-54.57,49.0,X-SC,1.58,82.0,-0.26,1.30,3.75,XY24,BTT,CEMENT
15,CAMS,4762.0,0.94,2.38,21.89,24.79,56443.0,5993.0,257849.0,-2.36,51.0,X-SC,1.87,86.0,0.11,1.81,26.07,X40N,NTT,MISC
66,SIEMENS,4671.5,0.26,-11.19,41.33,25.51,68308.0,-20820.0,165275.0,1.35,64.0,H-LC,2.03,49.0,-0.30,1.16,21.79,AR,ATH,ELECTRICAL
36,ICICIPRULI,790.0,0.14,2.89,27.29,30.97,49627.0,5112.0,181850.0,-21.26,55.0,X-MC,2.12,75.0,0.10,1.27,15.82,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-2.35,-37.06,114.82,35.20,92985.0,-47687.0,80983.0,6.14,19.0,X-SC,13.68,92.0,-0.51,0.57,0.00,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-0.59,-14.57,73.77,48.45,126367.0,-29208.0,171298.0,-20.73,28.0,X-MC,7.77,64.0,-0.23,1.20,0.00,X40N,ATH,FINANCE
54,PAGEIND,51605.07,-2.45,-8.68,38.06,26.07,56907.0,-14220.0,149520.0,-32.77,28.0,X-MC,9.94,55.0,-0.25,1.05,0.00,X40,ATH,APPARELS
55,PGHH,17907.65,0.34,-4.40,39.91,33.76,76621.0,-8835.0,191985.0,-32.51,35.0,X-MC,4.06,57.0,-0.12,1.35,0.57,X40,ATH,FMCG
19,COLPAL,3726.84,-0.63,-18.18,72.95,41.51,157200.0,-47875.0,215490.0,-2.56,40.0,X-MC,7.22,61.0,-0.30,1.51,1.12,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-0.91,-28.16,98.84,42.84,46158.0,-18306.0,46700.0,-53.17,41.0,X-SC,37.52,83.0,-0.40,0.33,1.62,XY24,NTT,MISC
60,RELAXO,1176.00,0.00,-47.00,187.35,52.31,144145.0,-68221.0,76939.0,-43.57,37.0,X-SC,4.81,91.0,-0.47,0.54,2.99,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-2.35,-37.06,114.82,35.20,92985.0,-47687.0,80983.0,6.14,19.0,X-SC,13.68,92.0,-0.51,0.57,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.10,-6.33,29.79,21.58,39107.0,-8865.0,131274.0,-52.80,35.0,X-MC,6.23,56.0,-0.23,0.92,16.22,X40N,ATH,IT
52,MEDANTA,1486.00,-0.28,3.75,19.60,24.08,25814.0,4755.0,131705.0,-4.69,47.0,X-SC,5.94,89.0,0.18,0.92,24.10,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-0.25,-14.96,40.26,19.28,118445.0,-51755.0,294201.0,-25.89,57.0,X-LC,7.02,1.0,-0.44,2.06,8.76,X40,ATH,IT
41,INFY,2275.00,0.52,6.76,45.07,54.87,150546.0,21139.0,334027.0,-16.73,62.0,X-LC,2.35,2.0,0.14,2.34,15.60,X40,BTT,IT
76,TMPV,600.00,1.70,-14.02,65.36,42.18,155339.0,-38743.0,237667.0,-23.08,35.0,X-LC,2.30,3.0,-0.25,1.67,2.95,XY24,NTT,AUTO
82,VBL,671.64,0.40,-2.33,38.91,35.67,120027.0,-7369.0,308473.0,-13.89,63.0,X-LC,5.48,4.0,-0.06,2.16,11.20,X40N,ATH,FMCG
43,ITC,452.00,-0.15,-1.38,11.98,10.44,23647.0,-2753.0,197385.0,-41.15,45.0,X-LC,1.32,5.0,-0.12,1.38,4.11,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-0.82,-14.47,112.40,81.67,90188.0,-13572.0,80238.0,7211.11,46.0,L-SC,19.36,271.0,-0.15,0.56,55.38,XR,NTT,CERAMICS
68,SONACOMS,806.63,-0.51,-11.92,58.43,39.54,52062.0,-12058.0,89101.0,-31.16,66.0,M-SC,6.69,220.0,-0.23,0.62,25.64,AR,ATH,AUTO
51,MASFIN,398.61,-0.08,-5.53,29.19,22.05,27020.0,-5415.0,92565.0,-18.70,49.0,H-SC,6.79,164.0,-0.20,0.65,34.55,XR,ATH,FINANCE
70,SURYODAY,216.00,0.95,-18.75,51.42,23.03,74818.0,-33568.0,145503.0,57.89,51.0,H-SC,9.71,167.0,-0.45,1.02,44.37,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,0.27,-8.57,9.39,0.02,14986.0,-14959.0,159600.0,143.30,67.0,M-SC,11.74,234.0,-1.00,1.12,38.82,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.27,-8.57,9.39,0.02,14986.0,-14959.0,159600.0,143.30,67.0,M-SC,11.74,234.0,-1.00,1.12,38.82,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-0.61,-0.14,21.12,20.95,53097.0,-361.0,251407.0,-1.19,70.0,X-LC,15.75,31.0,-0.01,1.76,35.44,X40,ATH,PAINTS
50,LTIM,7201.88,1.06,12.63,16.89,31.66,46827.0,31094.0,277245.0,8.32,69.0,H-LC,13.99,43.0,0.66,1.94,53.83,X200,ATH,IT
40,INDUSINDBK,1800.00,-1.29,-37.43,112.39,32.89,54293.0,-28898.0,48308.0,-696.43,60.0,L-MC,5.80,259.0,-0.53,0.34,33.04,XR,NTT,BANKS
32,HCLTECH,1922.01,0.91,6.37,17.27,24.73,44440.0,15399.0,257323.0,11.06,65.0,X-LC,4.89,7.0,0.35,1.80,22.57,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.96
1,25,44.85
2,50,75.74


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.43
MC    29.12
LC    25.46
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.73
X40      21.24
X40N     12.03
XR        9.62
XY25      9.08
AR        9.05
OX40N     7.60
X200      1.94
SR        0.97
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.89
X-MC    22.65
X-LC    20.16
M-SC    11.87
X-SC     8.27
H-MC     4.75
H-LC     3.10
L-SC     1.40
M-MC     1.38
M-LC     1.22
L-LC     0.98
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.83,-4.67,38.52
IT,13.87,-14.13,73.05
FINANCE,9.44,-15.01,63.60
MISC,7.41,-22.91,73.27
ELECTRICAL,6.01,-9.40,49.47
PAINTS,5.93,-9.11,25.32
INSURANCE,4.40,-0.48,35.89
PHARMA,3.89,-0.56,32.74
AUTO,3.40,-18.53,69.35


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3080343.0,21
XR,1290405.0,13
AR,1279708.0,10
X40,1005495.0,14
X40N,731760.0,9
OX40N,709195.0,10
XY25,344341.0,6
SR,280975.0,2
MH,77076.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3568618.0,25
M-SC,1367328.0,15
X-MC,1249057.0,16
X-LC,882801.0,11
X-SC,740757.0,8
H-MC,400335.0,3
L-SC,262116.0,3
M-LC,117923.0,1
H-LC,115135.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1222815.0      6
           AR         880852.0      5
M-SC       XY24       788163.0      6
H-SC       XR         777465.0      7
X-MC       X40        444491.0      7
X-LC       X40        390175.0      5
X-MC       XY24       374491.0      3
H-SC       OX40N      329435.0      4
X-SC       X40N       292310.0      3
M-SC       OX40N      292104.0      5
H-SC       SR         280975.0      2
X-SC       XY24       277618.0      3
X-MC       X40N       247484.0      4
X-LC       XY24       226672.0      2
H-MC       AR         209751.0      2
X-LC       X40N       191966.0      2
H-MC       XY24       190584.0      1
X-MC       XY25       182591.0      2
L-SC       XR         174460.0      2
X-SC       X40        170829.0      2
M-SC       XR         166264.0      2
           AR         120797.0      2
M-LC       XR         117923.0      1
L-SC       OX40N       87656.0      1
H-SC       MH          77076.0      1
X-LC       XY25        73988.0      2
H-LC       AR          68308.0      1
L-MC       XR          54293.0      1
M-MC       XY25        48619.0      1
H-LC       X200        46827.0      1
L-LC       XY25        39143.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
